<a href="https://colab.research.google.com/github/M-H-Amini/MachineLearningMiniCourse/blob/master/MLmini_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In The Name Of ALLAH
# Machine Learning *mini* Course
### PythonChallenge.ir
### Mohammad Hossein Amini (mhamini@aut.ac.ir)

# Logistic Regression

# Introduction

The theoretical stuff has been discussed in the video lectures. Let's implement a little...

First of all, we should import some modules.

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
print(tf.__version__)
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import glob
import math

# Discovering Sigmoid
Let's see **sigmoid** function in action.

In [0]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

xs = [i for i in np.arange(-10, 10, 0.5)]
ys = [sigmoid(x) for x in xs]

plt.figure()
plt.plot(xs, ys, 'r-')
plt.title(r'Sigmoid Function: $\frac{1}{1+e^{-x}}$')
plt.xlabel('X')
plt.ylabel('$\sigma(x)$')
plt.show()

# Importing Dataset
Today, we'll be using **graduate admission** dataset. It predicts the likelihood of being accepted in a graduate program. Let's import it in our program.

In [0]:
! unzip graduate\-admissions.zip

In [0]:
ds = pd.read_csv('Admission_Predict.csv')
ds = ds.drop('Serial No.', 1)
ds.head()

In [0]:
ds_arr = np.array(ds)
print(ds_arr.shape)

# Normalization
As discussed in the previous lecture, normalization helps a lot!

In [0]:
def normalize(ds):
  mean = np.array(ds.mean())
  mean = mean[np.newaxis, :]
  std = np.array(ds.std())
  std = std[np.newaxis, :]
  X = np.array(ds)
  X = (X-mean)/std
  return X

def invert(X, ds):
  mean = np.array(ds.mean())
  mean = mean[np.newaxis, :]
  std = np.array(ds.std())
  std = std[np.newaxis, :]
  X = (X*std) + mean
  return X

In [0]:
normalized_ds_arr = normalize(ds)
x = normalized_ds_arr[:, :-1]
y = normalized_ds_arr[:, -1:]

# Tensorflow Codes
Our implementation in tensorflow is much like the previous session.

In [0]:
X = tf.constant(x, dtype=tf.float32)
Y = tf.constant(y, dtype=tf.float32)

w = tf.Variable(np.random.rand(X.shape[1]+1, 1), dtype=tf.float32)

In [0]:
def h(x, w):
  x = tf.concat((tf.ones((x.shape[0], 1)), x), axis=1)
  return tf.sigmoid(tf.matmul(x,w))

def cost(x, y, w):
  return tf.reduce_mean(tf.losses.binary_crossentropy(y, h(x, w)))

In [0]:
optimizer = tf.optimizers.Adam()
w = tf.Variable(np.random.rand(X.shape[1]+1, 1), dtype=tf.float32)

def train_step(x, y, w, verbose=0):
  with tf.GradientTape() as t:
    J = cost(x, y, w)
  if verbose:
    print('Loss: {}'.format(J))
  w_grads = t.gradient(J, w)
  optimizer.apply_gradients(zip([w_grads], [w]))
  return w

print('Before: ', w)
w = train_step(X[15:16, :], Y[15:16, :], w, verbose = 1)
print('After: ', w)

In [0]:
def train(X, Y, max_iters=100, min_cost=0.01, w=None, verbose=0):
  if w is None:
    w = tf.Variable(np.random.rand(X.shape[1], 1), dtype=tf.float32)
  for i in range(max_iters):
    index = np.random.randint(0, X.shape[0])
    w = train_step(X[index:index+1, :], Y[index:index+1, :], w)
    if verbose:
      print('Cost: ', cost(X, Y, w).numpy())
  print("Training Done...")
  print("Cost: {}".format(cost(X, Y, w).numpy()))
  print("w = ", w)
  return w

w = train(X, Y, w=w, max_iters=1000, verbose=0)

In [0]:
mean = np.array(ds.mean())
mean = mean[np.newaxis, :]
std = np.array(ds.std())
std = std[np.newaxis, :]
o = h(X, w).numpy()
o = (o*std[0,-1])+mean[0,-1]
for i in range(50):
  print('No: {}'.format(i+1), '\tTarget: {}'.format(ds_arr[i, -1]), '\tPredicted: {}'.format(o[i, 0]))  